In [1]:
import tensorflow as tf
import os
os.environ["TF_USE_LEGACY_KERAS"]="1"
import keras
import tensorflow_model_optimization as tfmot
from sklearn.metrics.cluster import entropy
from tf_keras import layers,losses,optimizers,Sequential
from tf_keras.models import Model
from tf_keras.layers import Dense, Flatten, Dropout, GlobalAveragePooling2D,Input
from tf_keras.applications import VGG16
from tf_keras import datasets,models
import importlib

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [3]:
from importing_dataset import load_dataset
import Entropy
importlib.reload(Entropy)
train_examples, validation_examples, num_examples, num_classes, class_names = load_dataset('horses_or_humans', 70)

INFO:absl:Load dataset info from C:\Users\neyen\tensorflow_datasets\horses_or_humans\3.0.0
INFO:absl:Creating a tf.data.Dataset reading 2 files located in folders: C:\Users\neyen\tensorflow_datasets\horses_or_humans\3.0.0.
INFO:absl:Creating a tf.data.Dataset reading 1 files located in folders: C:\Users\neyen\tensorflow_datasets\horses_or_humans\3.0.0.
INFO:absl:Constructing tf.data.Dataset horses_or_humans for split ('train[:70%]', 'train[70%:]'), from C:\Users\neyen\tensorflow_datasets\horses_or_humans\3.0.0


In [4]:
from reformatting import reformat_image
BATCH_SIZE = 32
IMG_SIZE = (224, 224)
train_batches = train_examples.cache().shuffle(num_examples//4).map(reformat_image).batch(BATCH_SIZE).prefetch(1)
validation_batches = validation_examples.map(reformat_image).batch(BATCH_SIZE).prefetch(1)
train_batches

<_PrefetchDataset element_spec=(TensorSpec(shape=(None, 224, 224, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None,), dtype=tf.int64, name=None))>

In [7]:

from tf_keras import backend as K
# Assurez-vous que l'entrée est un KerasTensor
input_layer = K.placeholder(shape=(None, 224, 224, 3), dtype='float32')


# Bloc 1
x = layers.Conv2D(64, (3, 3), activation='relu', padding='same', name='block1_conv1')(input_layer)
x = layers.Conv2D(64, (3, 3), activation='relu', padding='same', name='block1_conv2')(x)
x = layers.MaxPooling2D((2, 2), strides=(2, 2), name='block1_pool')(x)

# Bloc 2
x = layers.Conv2D(128, (3, 3), activation='relu', padding='same', name='block2_conv1')(x)
x = layers.Conv2D(128, (3, 3), activation='relu', padding='same', name='block2_conv2')(x)
x = layers.MaxPooling2D((2, 2), strides=(2, 2), name='block2_pool')(x)

# Bloc 3
x = layers.Conv2D(256, (3, 3), activation='relu', padding='same', name='block3_conv1')(x)
x = layers.Conv2D(256, (3, 3), activation='relu', padding='same', name='block3_conv2')(x)
x = layers.Conv2D(256, (3, 3), activation='relu', padding='same', name='block3_conv3')(x)
x = layers.MaxPooling2D((2, 2), strides=(2, 2), name='block3_pool')(x)

# Bloc 4
x = layers.Conv2D(512, (3, 3), activation='relu', padding='same', name='block4_conv1')(x)
x = layers.Conv2D(512, (3, 3), activation='relu', padding='same', name='block4_conv2')(x)
x = layers.Conv2D(512, (3, 3), activation='relu', padding='same', name='block4_conv3')(x)
x = layers.MaxPooling2D((2, 2), strides=(2, 2), name='block4_pool')(x)

# Bloc 5
x = layers.Conv2D(512, (3, 3), activation='relu', padding='same', name='block5_conv1')(x)
x = layers.Conv2D(512, (3, 3), activation='relu', padding='same', name='block5_conv2')(x)
x = layers.Conv2D(512, (3, 3), activation='relu', padding='same', name='block5_conv3')(x)
x = layers.MaxPooling2D((2, 2), strides=(2, 2), name='block5_pool')(x)

# Ajouter des couches supplémentaires
x = layers.GlobalMaxPooling2D(name="global_max_pool")(x)
x = layers.Dropout(0.3, name="dropout")(x)
outputs = layers.Dense(2, activation="softmax", name="predictions")(x)

# Construire le modèle
model = Model(input_layer, outputs, name="custom_vgg16.keras")
model.summary()

# Charger les poids de VGG16
vgg16_base = VGG16(include_top=False, weights="imagenet", input_shape=(224, 224, 3))

from test_surgeon import Surgeon
# Appliquer les poids pour chaque couche manuellement
for layer, pretrained_layer in zip(model.layers, vgg16_base.layers):
    if isinstance(layer, layers.Conv2D):
        layer.set_weights(pretrained_layer.get_weights())

input_layer = model.input  # L'entrée du modèle
layer_0 = model.layers[1]  # Index 1 car la première couche est 'input_40'
surgeon = Surgeon(model)

# Si la couche 0 est une Conv2D, supprimer les canaux
if isinstance(layer_0, layers.Conv2D):
    # Surgeon pour supprimer les canaux
    surgeon.add_job('delete_channels', layer_0, channels=[0, 1, 2])
    pruned_model = surgeon.operate()

    # Redéfinir les entrées et sorties du modèle après modification
    pruned_model = Model(inputs=input_layer, outputs=pruned_model.output)

    # Redéfinir les entrées du modèle (pour qu'elles soient un KerasTensor valide)
    pruned_model.build(input_shape=(None, 224, 224, 3))

    # Recompiler le modèle pour qu'il soit prêt à l'entraînement ou à la prédiction
    pruned_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


else:
    print("La couche spécifiée n'est pas une couche Conv2D.")

pruned_model.summary()


Model: "custom_vgg16.keras"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)

Model: "model_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 61)      1708      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      35200     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0   

In [16]:
import Entropy_Keras_Surgeon
importlib.reload(Entropy_Keras_Surgeon)
from Entropy_Keras_Surgeon import EntropyPruningSurgeon
testt_model = models.load_model("model_experiments/custom_vgg16.keras")
testt_model.summary()
pruning = EntropyPruningSurgeon(model=testt_model, threshold=0.06)
pruningmodel = pruning.run()
pruningmodel.summary()

Model: "custom_vgg16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_40 (InputLayer)       [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)      

In [11]:
from tf_keras import layers, Model, Input
from tf_keras.applications import VGG16
from tf2kerassurgeon import Surgeon

# Initialisation du modèle
inputs = Input(shape=(224, 224, 3))

# Bloc 1
x = layers.Conv2D(64, (3, 3), activation='relu', padding='same', name='block1_conv1')(inputs)
x = layers.Conv2D(64, (3, 3), activation='relu', padding='same', name='block1_conv2')(x)
x = layers.MaxPooling2D((2, 2), strides=(2, 2), name='block1_pool')(x)

# Bloc 2
x = layers.Conv2D(128, (3, 3), activation='relu', padding='same', name='block2_conv1')(x)
x = layers.Conv2D(128, (3, 3), activation='relu', padding='same', name='block2_conv2')(x)
x = layers.MaxPooling2D((2, 2), strides=(2, 2), name='block2_pool')(x)

# Bloc 3
x = layers.Conv2D(256, (3, 3), activation='relu', padding='same', name='block3_conv1')(x)
x = layers.Conv2D(256, (3, 3), activation='relu', padding='same', name='block3_conv2')(x)
x = layers.Conv2D(256, (3, 3), activation='relu', padding='same', name='block3_conv3')(x)
x = layers.MaxPooling2D((2, 2), strides=(2, 2), name='block3_pool')(x)

# Bloc 4
x = layers.Conv2D(512, (3, 3), activation='relu', padding='same', name='block4_conv1')(x)
x = layers.Conv2D(512, (3, 3), activation='relu', padding='same', name='block4_conv2')(x)
x = layers.Conv2D(512, (3, 3), activation='relu', padding='same', name='block4_conv3')(x)
x = layers.MaxPooling2D((2, 2), strides=(2, 2), name='block4_pool')(x)

# Bloc 5
x = layers.Conv2D(512, (3, 3), activation='relu', padding='same', name='block5_conv1')(x)
x = layers.Conv2D(512, (3, 3), activation='relu', padding='same', name='block5_conv2')(x)
x = layers.Conv2D(512, (3, 3), activation='relu', padding='same', name='block5_conv3')(x)
x = layers.MaxPooling2D((2, 2), strides=(2, 2), name='block5_pool')(x)

# Ajouter des couches supplémentaires
x = layers.GlobalMaxPooling2D(name="global_max_pool")(x)
x = layers.Dropout(0.3, name="dropout")(x)
outputs = layers.Dense(2, activation="softmax", name="predictions")(x)

# Construire le modèle
model = Model(inputs, outputs, name="custom_vgg16.keras")

# Charger les poids de VGG16
vgg16_base = VGG16(include_top=False, weights="imagenet", input_shape=(224, 224, 3))

# Appliquer les poids pour chaque couche manuellement
for layer, pretrained_layer in zip(model.layers, vgg16_base.layers):
    if isinstance(layer, layers.Conv2D):
        layer.set_weights(pretrained_layer.get_weights())

# Appliquer le pruning sur la première couche de convolution
input_layer = model.input
layer_0 = model.layers[1]  # La première couche Conv2D dans votre modèle

surgeon = Surgeon(model)

# Vérifier si la couche est une Conv2D
if isinstance(layer_0, layers.Conv2D):
    surgeon.add_job('delete_channels', layer_0, channels=[0, 1, 2])
    pruned_model = surgeon.operate()

    # Redéfinir le modèle après pruning
    pruned_model = Model(inputs=input_layer, outputs=pruned_model.output)

    # Rebuild le modèle pour assurer que l'entrée est bien un KerasTensor
    pruned_model.build(input_shape=(None, 224, 224, 3))

    # Recompiler le modèle pour qu'il soit prêt à l'entraînement ou à la prédiction
    pruned_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

    # Affichage du résumé du modèle
    pruned_model.summary()

else:
    print("La couche spécifiée n'est pas une couche Conv2D.")


Deleting 3/64 channels from layer: block1_conv1


ValueError: All `inputs` values must be KerasTensors. Received: inputs=[<KerasTensor: shape=(None, 224, 224, 3) dtype=float32 (created by layer 'input_8')>] including invalid value KerasTensor(type_spec=TensorSpec(shape=(None, 224, 224, 3), dtype=tf.float32, name='input_8'), name='input_8', description="created by layer 'input_8'") of type <class 'tf_keras.src.engine.keras_tensor.KerasTensor'>

In [8]:
# Charger votre modèle
importlib.reload(Entropy)
from Entropy import EntropyPruning
testt_model = models.load_model("model_experiments/custom_vgg16.keras")
testt_model.summary()

NameError: name 'μ' is not defined

In [1]:
pruning = EntropyPruning(model=testt_model, threshold=0.01)
pruned_model = pruning.prune_model(None)
pruned_model.summary()

# # Test the model
# pruned_model.compile(
#     optimizer = 'adam',
#     loss = losses.SparseCategoricalCrossentropy(),
#     metrics=['accuracy'])
#
# pruned_model.evaluate(validation_batches)


NameError: name 'EntropyPruning' is not defined

In [8]:
import tensorflow as tf

from tf2kerassurgeon.operations import delete_channels
from tf2kerassurgeon import Surgeon

# Création d'un modèle de test
x = Input(shape=(224, 224, 3))
y = layers.Conv2D(32, (3, 3), activation='relu')(x)
y = layers.MaxPooling2D((2, 2))(y)
y = layers.Conv2D(64, (3, 3), activation='relu')(y)
y = layers.MaxPooling2D((2, 2))(y)
y = layers.Conv2D(64, (3, 3), activation='relu')(y)
y = layers.Flatten()(y)
y = layers.Dense(64, activation='relu')(y)
y = layers.Dense(10, activation='softmax')(y)

testt_model = Model(x, y)

# Vérification que la couche d'entrée est valide
input_layer = testt_model.input  # L'entrée du modèle
layer_0 = testt_model.layers[1]  # Index 1 car la première couche est 'input_40'
surgeon = Surgeon(testt_model)

# Si la couche 0 est une Conv2D, supprimer les canaux
if isinstance(layer_0, layers.Conv2D):
    # Surgeon pour supprimer les canaux
    surgeon.add_job('delete_channels', layer_0, channels=[0, 1, 2])
    pruned_model = surgeon.operate()

    # Redéfinir les entrées et sorties du modèle après modification
    pruned_model = Model(inputs=input_layer, outputs=pruned_model.output)

    # Redéfinir les entrées du modèle (pour qu'elles soient un KerasTensor valide)
    pruned_model.build(input_shape=(None, 224, 224, 3))

    # Recompiler le modèle pour qu'il soit prêt à l'entraînement ou à la prédiction
    pruned_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

    # Affichage du résumé du modèle
    pruned_model.summary()

else:
    print("La couche spécifiée n'est pas une couche Conv2D.")



Deleting 3/32 channels from layer: conv2d


ValueError: All `inputs` values must be KerasTensors. Received: inputs=[<KerasTensor: shape=(None, 224, 224, 3) dtype=float32 (created by layer 'input_1')>] including invalid value KerasTensor(type_spec=TensorSpec(shape=(None, 224, 224, 3), dtype=tf.float32, name='input_1'), name='input_1', description="created by layer 'input_1'") of type <class 'tf_keras.src.engine.keras_tensor.KerasTensor'>

In [38]:
import tensorflow as tf
import keras

from tf_keras import Model
from tf_keras.layers import Input, Conv2D, MaxPooling2D, Flatten, Dense
from tf2kerassurgeon.operations import delete_channels
from tf2kerassurgeon import Surgeon

# Création d'un modèle de test
x = Input(shape=(224, 224, 3))
y = Conv2D(32, (3, 3), activation='relu')(x)
y = MaxPooling2D((2, 2))(y)
y = Conv2D(64, (3, 3), activation='relu')(y)

# Ajouter une couche Dense pour la sortie
y = Flatten()(y)
y = Dense(64, activation='relu')(y)
y = Dense(10, activation='softmax')(y)

testt_model = Model(x, y)


layer_0 = testt_model.layers[1]  # Index 1 car la première couche est 'input_40'
# Vérification du type d'entrée
print(type(testt_model.input))  # Devrait afficher <class 'tensorflow.python.keras.engine.keras_tensor.KerasTensor'>

# Surgeon pour supprimer les canaux
surgeon = Surgeon(testt_model)
surgeon.add_job('delete_channels', layer_0, channels=[0, 1, 2])
pruned_model = surgeon.operate()


# Suppression des canaux 0, 1 et 2 de la couche Conv2D



# Créer une nouvelle entrée et sortie pour le modèle après suppression des canaux
pruned_model_input = pruned_model.input
pruned_model_output = pruned_model.output

# Créer un nouveau modèle pruné à partir des nouvelles entrées et sorties
pruned_model = Model(inputs=pruned_model_input, outputs=pruned_model_output)

# Recompiler le modèle pour s'assurer qu'il est prêt
pruned_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Affichage du résumé du modèle pour vérifier la suppression des canaux
pruned_model.summary()





<class 'tf_keras.src.engine.keras_tensor.KerasTensor'>
Deleting 3/32 channels from layer: conv2d_20


ValueError: All `inputs` values must be KerasTensors. Received: inputs=[<KerasTensor: shape=(None, 224, 224, 3) dtype=float32 (created by layer 'input_8')>] including invalid value KerasTensor(type_spec=TensorSpec(shape=(None, 224, 224, 3), dtype=tf.float32, name='input_8'), name='input_8', description="created by layer 'input_8'") of type <class 'tf_keras.src.engine.keras_tensor.KerasTensor'>

In [192]:
from tf_keras import optimizers
import tempfile
import tensorflow_model_optimization as tfmot
# Finish pruning after 2 epochs
epochs = 2
BATCH_SIZE = 32

def prune_model(model, initial_sparsity, final_sparsity, train_data=train_batches, val_data=validation_batches, epochs=epochs, feature_extractor=model ):

  # Create a tensorboard logfile
  logdir = tempfile.mkdtemp()
  # The end_step is the total number of iterations required for the training data which is basically the entire epochs over the length of the training data
  end_step = int(len(train_data) * epochs * 0.5)
  # Import the low-magnitude-pruning function
  prune_low_magnitude = tfmot.sparsity.keras.prune_low_magnitude

  # Set the prunung params
  pruning_params = {

      "pruning_schedule" : tfmot.sparsity.keras.PolynomialDecay(initial_sparsity=0,
                                                                final_sparsity=final_sparsity,
                                                                begin_step=0,
                                                                end_step=end_step)

                  }


  learning_rate_fn = optimizers.schedules.PolynomialDecay(
    0.001,
    1000,
    0.0001,
    power=0.5)

  # Model for pruning
  #feature_extractor = prune_low_magnitude(feature_extractor, **pruning_params)
  model_for_pruning = prune_low_magnitude(model, **pruning_params)

  # Recompile
  model_for_pruning.compile(optimizer= optimizers.Adam(),
                            loss= losses.SparseCategoricalCrossentropy(),
                            metrics=["accuracy"])
  #create callbacks
  callbacks = [tfmot.sparsity.keras.UpdatePruningStep(),
              tfmot.sparsity.keras.PruningSummaries(log_dir=logdir),
               #create_model_checkpoint(model_name=model.name),
              #tf.keras.callbacks.ReduceLROnPlateau(monitor="val_loss",
                                                  #patience=3,
                                                  #verbose=1),
               #tf.keras.callbacks.EarlyStopping(monitor="val_loss",
                                                        #patience=4,
                                                        #restore_best_weights=True)
                                                        ]

  # Fit the model
  model_for_pruning.fit(train_data,
                      validation_data=val_data,
                      batch_size=BATCH_SIZE,
                      epochs=epochs,
                      callbacks=callbacks)

  # Save the model
  #model_for_pruning.save(f"mnist_model_sparsity_{final_sparsity}")

  # Evaluate the model
  score = model_for_pruning.evaluate(val_data, verbose=0)
  metric_dict = {
      "sparsity" : final_sparsity,
      "val_loss" : np.round(score[0], 4),
      "val_accuracy" : np.round(score[1] * 100, 4)
  }
  return logdir, metric_dict, model_for_pruning

In [203]:
# Choose the best sparsity level
final_sparsity = 0.80

logdir, metrics, pruned_model = prune_model(model=pruned_model,
          initial_sparsity=0,
          final_sparsity=final_sparsity,
          epochs=3)

Epoch 1/3
23/23 [==============================] - 159s 7s/step - loss: 0.6922 - accuracy: 0.5257 - val_loss: 0.6947 - val_accuracy: 0.4838
Epoch 2/3
23/23 [==============================] - 174s 8s/step - loss: 0.6920 - accuracy: 0.5257 - val_loss: 0.6949 - val_accuracy: 0.4838
Epoch 3/3
23/23 [==============================] - 145s 6s/step - loss: 0.6920 - accuracy: 0.5257 - val_loss: 0.6949 - val_accuracy: 0.4838


In [196]:
model_for_export = tfmot.sparsity.keras.strip_pruning(pruned_model)

_, pruned_keras_file = tempfile.mkstemp('.h5')
models.save_model(model_for_export, pruned_keras_file, include_optimizer=False)
print('Saved pruned Keras model to:', pruned_keras_file)

C:\Users\neyen\AppData\Local\Temp\ipykernel_71452\369930696.py:4: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native TF-Keras format, e.g. `model.save('my_model.keras')`.
  models.save_model(model_for_export, pruned_keras_file, include_optimizer=False)


Saved pruned Keras model to: C:\Users\neyen\AppData\Local\Temp\tmpjh1lij65.h5
